In [1]:
%cd ..

/home/anthony/workspace/yt-data


In [2]:

import tensorflow as tf
import os
from typing import List, Tuple, Iterator
from rangedict import RangeDict
import numpy as np

from thumbs.experiment import Experiment
from thumbs.loss import Loss
from thumbs.data import get_pokemon_data256
from thumbs.params import HyperParams, MutableHyperParams
from thumbs.model.model import Model, BuiltModel

from keras.models import Sequential
from keras.layers import Dense, Reshape, Conv2DTranspose, Flatten, LeakyReLU
from keras.layers import (
    Activation,
    BatchNormalization,
    Dense,
    GaussianNoise,
    Dropout,
    Flatten,
    Reshape,
    ReLU,
    LeakyReLU,
    LayerNormalization,
    # BatchNormalizationV2,
)

# from keras.layers.normalization.batch_normalization_v1 import (
#     BatchNormalization,
# )
from tensorflow.compat.v1.keras.layers import BatchNormalization as BatchNormalizationV1
from keras.layers.convolutional import Conv2D, Conv2DTranspose

from thumbs.train import Train, TrainMSE, TrainBCE, TrainBCESimilarity, TrainWassersteinGP

2023-06-29 20:19:09.852307: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/anthony/workspace/yt-data/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [39]:
ndf = 64 + 32

def build_discriminator(img_shape=(128,128,3)):

    model = Sequential(name="discriminator")

    model.add(Conv2D(ndf, kernel_size=2, strides=2,  input_shape=img_shape))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2D(ndf*2, kernel_size=2, strides=1))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2D(ndf*3, kernel_size=5, strides=2))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2D(ndf*4, kernel_size=5, strides=5))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2D(ndf*5, kernel_size=6, strides=6))
    model.add(Flatten())
    model.add(Dense(1))


    model.summary()
    return model

build_discriminator()

Model: "discriminator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_195 (Conv2D)         (None, 64, 64, 96)        1248      
                                                                 
 leaky_re_lu_170 (LeakyReLU)  (None, 64, 64, 96)       0         
                                                                 
 conv2d_196 (Conv2D)         (None, 63, 63, 192)       73920     
                                                                 
 leaky_re_lu_171 (LeakyReLU)  (None, 63, 63, 192)      0         
                                                                 
 conv2d_197 (Conv2D)         (None, 30, 30, 288)       1382688   
                                                                 
 leaky_re_lu_172 (LeakyReLU)  (None, 30, 30, 288)      0         
                                                                 
 conv2d_198 (Conv2D)         (None, 6, 6, 384)       